In [ ]:
!pip install ultralytics

In [8]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.9/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
    data= "/content/drive/MyDrive/newSplitDataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="custom_tree_detector",
    project="runs/train"
)
print("✅ Training complete.")
print(results)

In [10]:
import os

base_dir = 'runs/train'

if os.path.exists(base_dir):
    runs = [os.path.join(base_dir, d) for d in os.listdir(base_dir)]
    latest_run = max(runs, key=os.path.getmtime)
    print("🟢 Latest training run folder:", latest_run)
else:
    print("❌ No training runs found.")


🟢 Latest training run folder: runs/train/custom_tree_detector


In [ ]:
import os

# Replace this with your actual training folder path
output_folder = 'runs/train/custom_tree_detector'

# List contents of the folder
files = os.listdir(output_folder)
print("📁 Contents of the training folder:")
for f in files:
    print(" -", f)


In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# ==== CONFIGURATION ====
img_path = "/content/drive/MyDrive/splitted_dataset_forYOLOv8/images/train/DJI_0128.jpg"
label_path = "/content/drive/MyDrive/splitted_dataset_forYOLOv8/labels/train/DJI_0128.txt"
output_path = "/content/annotated_image.jpg"

# ==== LOAD IMAGE ====
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError(f"Image not found: {img_path}")
h, w = img.shape[:2]

# ==== LOAD ANNOTATIONS ====
with open(label_path, 'r') as f:
    lines = f.readlines()

# ==== DRAW BOUNDING BOXES ====
for line in lines:
    parts = line.strip().split()
    if len(parts) != 5:
        continue

    class_id, xc, yc, bw, bh = map(float, parts)
    x1 = int((xc - bw / 2) * w)
    y1 = int((yc - bh / 2) * h)
    x2 = int((xc + bw / 2) * w)
    y2 = int((yc + bh / 2) * h)

    # Draw rectangle
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Optionally put class label
    cv2.putText(img, f"Class {int(class_id)}", (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)



    # Show image
cv2_imshow(img)


# ==== SAVE ANNOTATED IMAGE ====
cv2.imwrite(output_path, img)
print()
print(f"✅ Annotated image saved to: {output_path}")

In [ ]:
from ultralytics import YOLO
import os

# ==== CONFIGURATION ====
model_path = "/content/runs/train/custom_tree_detector/weights/best.pt"
frames_folder = "/content/drive/MyDrive/splitted_dataset_forYOLOv8/images/train"
output_folder = "/content/inference_output"

# ==== LOAD MODEL ====
model = YOLO(model_path)

# ==== RUN INFERENCE ====
results = model.predict(
    source=frames_folder,
    save=True,
    save_txt=True,
    project=output_folder,
    name='run'
)

print("✅ Inference completed. Check:", os.path.join(output_folder, 'run'))
